In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
%matplotlib inline  
from sklearn.model_selection import StratifiedKFold
from joblib import load, dump
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from torchvision import models as md
from torch import nn
from torch.nn import functional as F
import re
import math
import collections
from functools import partial
from torch.utils import model_zoo
from sklearn import metrics
from collections import Counter
import json
import torch.optim as optim
from torchvision import transforms
from torch.optim import lr_scheduler
from torch import nn
from torch.nn import functional as F
from sklearn.model_selection import KFold
from PIL import Image, ImageFile
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
import cv2
import gc
from statsmodels.stats.weightstats import zconfint
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic
from math import sqrt
from scipy import stats
import random
import time
import torchvision

In [ ]:
base_image_dir = os.path.join('..', 'input/aptos2019-blindness-detection/')
train_dir = os.path.join(base_image_dir,'train_images/')
df = pd.read_csv(os.path.join(base_image_dir, 'train.csv'))
df['path'] = df['id_code'].map(lambda x: os.path.join(train_dir,'{}.png'.format(x)))
df = df.sample(frac=1).reset_index(drop=True) #shuffle dataframe

In [ ]:
# удаляем дубликаты
duplicates = pd.read_csv('../input/duplicates/duplicates.csv', sep=';')
for j in duplicates['id_code']:
    df = df.drop(df[df['id_code']==j].index)
    
df = df.drop(columns=['id_code'])

In [ ]:
DEBUG = False

# TO DO: Random oversample / undersample for each fold (model)
# TO DO: use old dataset for random oversample


base_image_dir2 = os.path.join('..', 'input/retinopathytrain2015/retinopathy-train-2015/')
train_dir = os.path.join(base_image_dir2,'rescaled_train_896/')
df_2015 = pd.read_csv('../input/retinopathytrain2015/retinopathy-train-2015/trainLabels.csv')
df_2015['path'] = df_2015['image'].map(lambda x: os.path.join(train_dir,'{}.png'.format(x)))
df_2015 = df_2015.drop(columns=['image'])
df_2015 = df_2015.sample(frac=1).reset_index(drop=True) #shuffle dataframe
df_2015.columns = ['diagnosis', 'path']
df_2015_decr = df_2015.drop(df_2015.loc[df_2015['diagnosis']==0].sample(frac=0.9).index) #выкидываем 0.7 0 диагноза

TrainData = df
TrainData = TrainData.reset_index(drop=True)

if DEBUG:
    TrainData = TrainData[:100]

In [ ]:
delete = ['../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5178_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/13222_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2107_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/11855_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38478_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/14410_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43339_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38324_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/12329_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/7788_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39130_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/11910_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/21345_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/16581_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/8681_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/21297_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27519_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/17221_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5803_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/15387_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/11734_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4043_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/13254_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/15631_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31327_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/14467_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/42106_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/24881_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41445_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/35861_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/10131_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/33174_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2804_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/21529_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3307_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/9752_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/33646_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40341_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/14511_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/35704_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38518_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/18325_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/42498_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4593_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4583_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39106_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38447_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/11023_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/6833_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/22568_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/12735_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/9831_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/26863_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3838_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/23248_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/33287_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36539_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36006_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/21443_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/42832_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36856_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/33832_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/10567_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41301_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41032_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/22048_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/20842_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/14915_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/10844_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/10645_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43802_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41031_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/7378_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3797_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/28772_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/19600_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/20878_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/7853_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27452_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40163_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4688_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30658_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/13923_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43921_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4773_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/11249_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/18213_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41903_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/13588_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39574_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/10976_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/8363_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/21345_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/13051_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/24227_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31945_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32674_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/17221_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/378_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40727_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/8588_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/16085_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/19663_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/26486_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/8652_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/8677_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25083_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/26929_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/6337_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3381_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/23814_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/10043_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/28569_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/19285_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/14139_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/26877_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/24437_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/26863_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/14129_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/20742_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25765_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4300_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/9608_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43694_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/10427_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/10321_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/19467_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/24207_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32422_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/7869_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25608_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/33832_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/44247_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/11750_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/11050_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43694_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25428_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/14724_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38325_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/28086_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4086_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41519_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/21511_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/13180_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41485_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43839_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37813_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2516_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38831_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31659_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43339_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/21345_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/29139_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/6785_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/29139_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/6785_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/29492_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/15408_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32993_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40523_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/19177_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/326_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36627_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32401_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/42570_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27885_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37611_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40499_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38083_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25077_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5296_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/12872_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39999_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43074_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/22906_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41342_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/8470_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40447_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41201_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40405_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/23648_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43560_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/13251_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31497_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27960_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2810_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/22048_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/13901_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/375_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4831_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/22048_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/375_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38507_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31183_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/16448_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/21297_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/22767_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/24980_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5250_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30354_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32541_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/8744_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/54_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/54_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/79_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/172_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/184_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/184_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/219_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/283_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/283_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/378_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/391_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/466_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/466_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/670_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/670_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/679_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/679_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/766_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/766_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1037_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1119_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1138_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1138_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1178_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1362_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1362_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1496_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1496_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1636_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1640_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1640_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1675_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1675_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1776_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1776_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1893_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1893_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1926_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1926_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1933_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1965_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/1965_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2010_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2010_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2011_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2107_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2180_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2180_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2194_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2357_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2357_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2531_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2531_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2610_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2610_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2668_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2903_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/2903_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3029_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3029_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3194_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3213_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3213_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3267_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3267_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3271_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3271_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3275_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3303_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3371_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3379_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3379_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3397_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3397_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3501_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3525_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3554_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3554_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3562_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3570_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3570_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3587_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3587_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3611_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3636_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3738_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3780_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3780_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3829_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3993_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/3993_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4005_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4005_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4307_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4307_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4443_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4559_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4561_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4561_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4601_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4601_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4674_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4674_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4764_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4831_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4839_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4931_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4931_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4965_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/4965_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5019_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5100_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5156_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5156_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5178_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5185_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5185_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5258_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5258_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5275_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5406_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5410_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5410_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5441_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5530_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5632_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5632_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5795_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5877_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5877_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5927_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5927_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5947_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/5953_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/6014_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/6014_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/6299_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/6299_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/6544_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/6684_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/6684_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/6691_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/6707_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/7383_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/7383_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/7607_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/8116_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/8116_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/8342_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/8583_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/8669_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/8677_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/8681_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/8740_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/8740_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/8825_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/9116_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/9205_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/9205_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/9353_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/9353_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/9450_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/9450_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/9590_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/9590_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/10043_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/10234_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/10853_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/11101_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/11196_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/11647_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/11647_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/11648_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/11649_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/11648_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/12101_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/12575_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/12630_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/12676_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/12699_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/12861_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/13022_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/13022_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/13534_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/13534_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/13584_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/13584_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/13627_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/13627_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/13780_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/13786_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/14139_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/14160_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/14212_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/14474_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/14807_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/14807_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/15038_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/15038_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/15222_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/15302_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/15590_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/15954_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/16581_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/16802_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/16937_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/17139_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/17139_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/17411_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/17426_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/17555_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/17555_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/17925_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/18017_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/18030_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/18819_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/18862_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/18901_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/18972_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/19047_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/19278_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/19347_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/19811_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/20093_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/20146_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/20402_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/20677_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/20677_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/20742_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/20824_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/21297_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/21345_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/21373_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/21444_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/21513_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/21720_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/21757_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/21788_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/21788_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/22026_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/22665_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/22665_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/22687_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/22687_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/22716_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/22716_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/22760_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/22778_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/22906_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/22978_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/23284_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/23284_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/23325_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/23325_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/23433_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/23433_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/23451_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/23540_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/23692_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/23761_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/23761_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/23957_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/24074_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/24074_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/24437_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/24472_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/24478_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/24657_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/24657_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/24772_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/24780_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25154_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25154_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25166_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25166_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25181_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25181_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25198_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25198_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25291_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25385_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25385_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25588_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25620_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/25751_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/26076_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/26190_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/26300_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/26300_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/26459_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/26459_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/26541_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/26659_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/26863_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27081_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27081_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27100_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27452_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27506_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27506_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27581_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27597_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27597_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27674_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27674_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27727_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27727_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27803_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27838_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27838_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27850_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/27960_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/28000_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/28072_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/28072_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/28077_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/28172_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/28172_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/28369_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/28369_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/28387_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/28549_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/28722_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/28772_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/28976_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/29018_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/29018_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/29020_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/29020_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/29027_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/29027_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/29418_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/29418_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/29514_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/29560_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/29599_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/29705_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/29762_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/29960_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/29960_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30000_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30039_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30039_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30226_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30226_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30255_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30274_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30277_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30277_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30303_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30567_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30591_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30591_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30675_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30723_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30723_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30747_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/30747_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31034_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31160_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31183_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31202_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31421_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31421_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31497_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31635_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31635_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31647_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31647_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31659_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31837_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31837_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31924_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31924_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31945_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31994_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/31994_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32064_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32149_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32176_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32220_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32230_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32230_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32253_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32311_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32311_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32320_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32320_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32486_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32523_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32550_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32591_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32637_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32668_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32668_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32674_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32773_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32870_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32979_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/32979_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/33080_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/33287_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/33304_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/33319_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/33319_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/33337_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/33337_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/33414_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/33414_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/33646_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/33820_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/33891_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/34008_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/34101_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/34241_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/34241_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/34284_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/34368_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/34442_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/34442_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/34470_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/34470_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/34584_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/34584_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/34731_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/34731_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/34992_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/34992_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/34995_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/34995_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/35067_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/35092_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/35112_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/35112_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/35215_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/35394_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/35621_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/35637_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/35637_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/35668_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/35766_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/35766_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/35863_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/35863_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/35962_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/35966_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36006_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36099_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36184_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36190_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36190_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36256_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36426_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36426_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36453_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36453_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36502_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36502_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36635_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36635_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36640_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36640_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36653_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36653_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36793_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36793_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36903_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/36903_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37035_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37064_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37079_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37117_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37192_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37192_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37345_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37354_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37354_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37422_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37422_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37611_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37636_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37636_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37773_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37773_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37783_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37834_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37834_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37846_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37867_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37868_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37962_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/37962_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38083_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38116_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38163_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38186_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38186_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38324_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38325_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38326_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38373_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38481_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38481_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38513_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38513_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38518_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38668_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38668_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38795_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38856_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38856_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/38907_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39044_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39044_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39081_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39081_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39106_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39134_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39134_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39353_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39353_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39365_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39365_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39459_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39505_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39505_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39530_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39530_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39588_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39632_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39741_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39741_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39757_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39757_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39772_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39983_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39994_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/39999_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40089_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40178_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40178_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40183_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40183_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40189_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40259_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40319_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40323_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40341_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40499_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40557_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40616_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40937_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40937_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40994_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/40994_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41021_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41201_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41301_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41387_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41387_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41419_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41419_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41440_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41445_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41499_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41499_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41561_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41566_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41566_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41746_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41746_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41761_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41761_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/41903_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/42074_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/42074_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/42082_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/42119_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/42119_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/42484_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/42542_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/42891_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43025_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43062_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43062_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43119_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43166_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43379_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43379_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43457_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43490_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43563_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43589_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43589_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43593_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43593_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43594_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43688_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43929_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43932_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43985_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/43998_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/44100_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/44148_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/44189_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/44193_left.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/44193_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/44247_right.png',
'../input/retinopathytrain2015/retinopathy-train-2015/rescaled_train_896/44249_right.png']

In [ ]:
for path in df_2015_decr['path'].reset_index(drop=True):
    for bad in delete:
        if path == bad:
            df_2015_decr = df_2015_decr.drop(df_2015_decr[df_2015_decr['path']==bad].index)

In [ ]:
df_2015_decr = df_2015_decr.reset_index(drop=True)

In [ ]:
def get_random_df(df):
    randomdf = df[df['diagnosis']==0][:2000].sample(frac=random.random())
    randomdf = randomdf.append(df[df['diagnosis']==1].sample(frac=random.random()))
    randomdf = randomdf.append(df[df['diagnosis']==2][:2000].sample(frac=random.random()))
    randomdf = randomdf.append(df[df['diagnosis']==3].sample(frac=random.random()))
    randomdf = randomdf.append(df[df['diagnosis']==4].sample(frac=random.random()))
    randomdf = randomdf.sample(frac=1).reset_index(drop=True) 
    
    return randomdf

In [ ]:
# Get random sample
#df_2015_decr = get_random_df(df_2015_decr)

In [ ]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
IMAGE_SIZE = 256 # 224?
def info_image(im):
    # Compute the center (cx, cy) and radius of the eye
    cy = im.shape[0]//2
    midline = im[cy,:]
    midline = np.where(midline>midline.mean()/3)[0]
    if len(midline)>im.shape[1]//2:
        x_start, x_end = np.min(midline), np.max(midline)
    else: # This actually rarely happens p~1/10000
        x_start, x_end = im.shape[1]//10, 9*im.shape[1]//10
    cx = (x_start + x_end)/2
    r = (x_end - x_start)/2
    return cx, cy, r

def resize_image(im, augmentation=True):
    # Crops, resizes and potentially augments the image to IMAGE_SIZE
    cx, cy, r = info_image(im)
    scaling = IMAGE_SIZE/(2*r)
    rotation = 0
    if augmentation:
        scaling *= 1 + 0.3 * (np.random.rand()-0.5)
        rotation = 360 * np.random.rand()
    M = cv2.getRotationMatrix2D((cx,cy), rotation, scaling)
    M[0,2] -= cx - IMAGE_SIZE/2
    M[1,2] -= cy - IMAGE_SIZE/2
    return cv2.warpAffine(im,M,(IMAGE_SIZE,IMAGE_SIZE)) # This is the most important line

def subtract_median_bg_image(im):
    k = np.max(im.shape)//20*2+1
    bg = cv2.medianBlur(im, k)
    return cv2.addWeighted (im, 4, bg, -4, 128)

def subtract_gaussian_bg_image(im):
    k = np.max(im.shape)/10
    bg = cv2.GaussianBlur(im ,(0,0) ,k)
    return cv2.addWeighted (im, 4, bg, -4, 128)

def id_to_image(path, resize=True, augmentation=False, subtract_gaussian=False, subtract_median=False):
    #path = '../input/train_images/{}.png'.format(id_code)
    im = cv2.imread(path)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    if resize_image:
        im = resize_image(im, augmentation)
        
    randint = np.random.randint(1,4)
    
    if randint == 1:
        im = subtract_gaussian_bg_image(im)
    if randint == 2:
        im = subtract_gaussian_bg_image(im)
    if randint == 3:
        pass
        
    return im

In [ ]:
def crop_image_from_gray(img,tol=7):
    """
    Crop out black borders
    https://www.kaggle.com/ratthachat/aptos-updated-preprocessing-ben-s-cropping
    """  
    
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        mask = gray_img>tol        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0):
            return img
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img


def circle_crop(img):   
    """
    Create circular crop around image centre    
    """    
    
    img = cv2.imread(img)
    img = crop_image_from_gray(img)    
    
    height, width, depth = img.shape    
    
    x = int(width/2)
    y = int(height/2)
    r = np.amin((x,y))
    
    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)
    
    return img 

In [ ]:
def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [ ]:
class RetinopathyDatasetTrain(Dataset):

    def __init__(self, df, transform):

        self.data = df
        self.data = self.data.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.loc[idx, 'path']
        image = id_to_image(img_name, resize=True, augmentation=True, subtract_gaussian=False, subtract_median=False)
        transform_to_pil = torchvision.transforms.Compose([torchvision.transforms.ToPILImage(mode=None)])
        image = transform_to_pil(image)

#         image = cv2.imread(img_name)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         image = crop_image_from_gray(image)
#         #image = resize_image(image, augmentation=False)
#         image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
#         image = cv2.addWeighted( image,4, cv2.GaussianBlur( image , (0,0) , 30) ,-4 ,128)
#         image = transforms.ToPILImage()(image)
    
        #image = Image.open(img_name)
        #image = image.resize((IMAGE_SIZE, IMAGE_SIZE), resample=Image.BILINEAR)
        
        if self.transform != None:
            image = self.transform(image)
            
        label = torch.tensor(self.data.loc[idx, 'diagnosis'])
        return {#'image': transforms.ToTensor()(image),
                'image': image,
                'labels': label,
               }

In [ ]:
import albumentations
from albumentations import torch as AT

data_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation((-120, 120)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

data_transform_test = transforms.Compose([
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomRotation((-120, 120)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:
def get_model(model_type='resnext', out=1):
    # TO DO: Add more models!
    if model_type=='resnext':
        model = torchvision.models.resnext101_32x8d(pretrained=True)
        model.fc = nn.Linear(in_features=2048, out_features=out, bias=True)
        return model

In [ ]:
from sklearn.metrics import cohen_kappa_score
def quadratic_kappa(y_hat, y):
    return torch.tensor(cohen_kappa_score(np.around(y_hat), y, weights='quadratic'),device='cuda:0')

In [ ]:
# Использовать на рандомизированных предсказаниях, для подборки трешхолда
import scipy as sp
from functools import partial
from sklearn import metrics
from collections import Counter
import json

class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = [0.5, 1.5, 2.5, 3.5]

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4

        ll = metrics.cohen_kappa_score(y, X_p, weights='quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')
        print(-loss_partial(self.coef_['x']))

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        return self.coef_['x']

In [ ]:
num_epochs = 20
batch_size = 20 # 25
kf = KFold(n_splits=5)
device = torch.device("cuda:0")
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
fold = 0
NUM_FOLDS = 1
optR = OptimizedRounder()

TrainData = TrainData.append(df_2015_decr)
TrainData = TrainData.sample(frac=1).reset_index(drop=True) #shuffle dataframe

for train_index, test_index in kf.split(TrainData):
    if fold > 0 and fold > NUM_FOLDS-1:
        print('Brake')
        break
    fold +=1
    validation_kappa = []
    validation_score = []
    seed_everything(123+fold)

    gc.collect()
    print('=' * 10)
    #model = EfficientNet.from_pretrained('efficientnet-b5', num_classes=1)
    model = get_model(model_type='resnext')
    model = model.to(device)
    #plist = [{'params': model._fc.parameters(), 'lr': 1e-3}]
    plist = [
             {'params': model.layer4.parameters(), 'lr': 1e-3},
             {'params': model.fc.parameters(), 'lr': 1e-3}
             ]
    criterion = nn.MSELoss()
    optimizer = optim.Adam(plist, lr=0.0002)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)
    #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,patience=2,verbose=1,factor=0.5)
    
    since = time.time()
    
    valid_dataset = RetinopathyDatasetTrain(TrainData.loc[test_index,:], transform=data_transform_test)
    data_valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    # TO DO: Random oversampling / undersampling for different folds, preferably on fly.
    train_dataset = RetinopathyDatasetTrain(TrainData.loc[train_index,:], transform=data_transform)
    data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    
    epoch_kappa_es = 0 # initial values for early stopping
    epoch_loss_eval_es = 10000 # initial values for early stopping
    counter_es = 0
    saving = False #initial value
    
    for epoch in range(num_epochs):
        since_epoch = time.time()
        print('-' * 10)
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        
        ##################################################
        #TRAIN
        ##################################################
        if epoch == 2:
            print('Now training all layers')
            optimizer = optim.Adam(model.parameters(), lr=0.0001)
            #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,patience=2,verbose=1)
            
        model.train()
        running_loss = 0.0
        # TO DO: Early stoping
        # TO DO: Random oversampling / undersampling for different folds ON FLY! To save evaluation data
        for i, d in enumerate(data_loader):
            inputs = d["image"]
            labels = d["labels"].view(-1, 1)
            inputs = inputs.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                # https://medium.com/huggingface/training-larger-batches-practical-tips-on-1-gpu-multi-gpu-distributed-setups-ec88c3e51255
                #loss = loss / accumulation_steps                # Normalize our loss (if averaged)
                loss.backward() ###
#                 if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
#                     optimizer.step()                            # Now we can do an optimizer step
#                     model.zero_grad()
                optimizer.step() ###
            running_loss += loss.item() * inputs.size(0)
            gc.collect()
            
        epoch_loss = running_loss / len(train_index)
        
        ##################################################
        #VALIDATION
        ##################################################
        # TO DO: TTA for Evaluation - low rank
        model.eval()

        test_preds = np.zeros(len(test_index)) # для предсказаний
        labels_preds = np.zeros(len(test_index))  # для тру лейблов
        running_loss = 0.0
        for i, d in enumerate(data_valid_loader):
            inputs = d["image"]
            labels = d["labels"].view(-1, 1)
            inputs = inputs.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)
            
            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            test_preds[i * batch_size:(i + 1) * batch_size] = optR.predict(outputs.cpu().numpy().flatten(),[0.5, 1.5, 2.5, 3.5])
            labels_preds[i * batch_size:(i + 1) * batch_size] = labels.cpu().numpy().flatten()

        epoch_loss_eval = running_loss / len(test_index)
        epoch_kappa = quadratic_kappa(test_preds, labels_preds)
        
        time_elapsed_epoch = time.time() - since_epoch
        
        print('time={:.2f}s Training Loss={:.4f} Test Loss={:.4f} Eval kappa={:.4f}'.format(time_elapsed_epoch, epoch_loss, epoch_loss_eval, epoch_kappa))
        scheduler.step(epoch_loss_eval)
        
        if epoch == 5: 
            saving = True
        
        # Если лосс улучшился - сохраняем / перезаписываем
        if epoch_loss_eval < epoch_loss_eval_es:
            epoch_loss_eval_es = epoch_loss_eval
            counter_es = 0
            if saving:
                print('saving model')
                torch.save(model.state_dict(), "model {}.bin".format(fold))
        
        # Если лосс не улучшился, то увеличиваем счетчик на 1
        if epoch_loss_eval > epoch_loss_eval_es:
            counter_es += 1
            print('counter={}'.format(counter_es))
            
        
        # Если счетчик достигает 5, то прерываем обучение
#         if counter_es == 10:
#             print('early stoping,', counter_es, epoch_kappa_es)
#             break
            
    validation_kappa.append(epoch_kappa)
    validation_score.append(epoch_loss_eval)
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    torch.save(model.state_dict(), "model final {}.bin".format(fold))
    del model
    gc.collect()

# TO DO: Final evaluation using all models (especially with random oversampling/undersampling)
# TO DO: использовать все фолды как выборку. провести перестановочный тест для определения лучше текущая модель,
# чем предыдущая или нет

In [ ]:
from IPython.display import FileLinks
FileLinks('.') # input argument is specified folder